# Required Modules
Set up and install the necessary modules for solving Sokoban puzzles.

In [1]:
# Install the pysat module for SAT-based problem-solving.
!pip install python-sat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 46.8 MB/s eta 0:00:00


In [2]:
# Import necessary modules for SAT encoding and decoding.
from pysat.formula import CNF
from pysat.solvers import Solver
import copy

# Atom Encoding and Decoding
Implement methods to encode and decode puzzle elements.

In [3]:
# Define global dictionaries to map atoms to encoded integers.
atoms_dict = {} # (kind, i, j, t) => ID
reverse_atoms_dict ={} # ID => (kind, i, j, t)
lastUsedID = 0

In [4]:
# Function to encode variables as unique positive integers.
def encodeAtom(kind, i, j, t):
        global lastUsedID, atoms_dict, reverse_atoms_dict

        if not ((kind, i, j, t) in atoms_dict):
          lastUsedID += 1
          atoms_dict[(kind, i, j, t)] =  lastUsedID
          reverse_atoms_dict[lastUsedID] = (kind, i, j, t)
        return atoms_dict[(kind, i, j, t)]

Finally, decode the numerical IDs back into their respective atom representations.

In [5]:
# Function to decode integers back to their corresponding atoms.
def decodeAtom(ID):
    global reverse_atoms_dict
    if abs(ID) in reverse_atoms_dict:
        return reverse_atoms_dict[abs(ID)]
    return -1

# Puzzle Conditions
Define logical constraints to guide the solution.

## 1. Initial Board Configuration
Define clauses representing the puzzle's initial state.

In [6]:
# Generate clauses for the initial state of the puzzle board.
def create_initial_clauses(clauses, StartBoard):

    height = len(StartBoard)
    width = len(StartBoard[0])

    for i in range(height):
      for j in range(width):

        if StartBoard[i][j] == 'b':
          clauses.append([encodeAtom(1, i, j, 0)])
        else:
          clauses.append([-encodeAtom(1, i, j, 0)])
        if StartBoard[i][j] == 'p':
          clauses.append([encodeAtom(0, i, j, 0)])
        else:
          clauses.append([-encodeAtom(0, i, j, 0)])

    return clauses

## 2. Final Board Configuration
Add constraints to meet the puzzle's goal conditions.

In [7]:
# Add constraints to ensure the final board satisfies goal conditions.
def create_Final_clause(clauses, goal_position, max_steps):

  clauses.append([encodeAtom(1, *goal_position, max_steps)])
  return clauses

## Remove final clause (if steps were not enough)
def Delete_Final_clause(clauses, goal_position, max_steps):

  clauses.clauses.pop()

  return clauses

## 3. State Transitions
Model valid transitions and movements within the puzzle.

In [8]:

# Represent valid transitions between puzzle states using clauses.
def create_transition_clauses(clauses, StartBoard, step_number):
    height = len(StartBoard)
    width = len(StartBoard[0])
    k = step_number

    for i1 in range(height):
      for i2 in range(height):
        for j1 in range(width):
          for j2 in range(width):
            if i1 != i2 or j1 != j2:
              clauses.append([-encodeAtom(0, i1, j1, k), -encodeAtom(0, i2, j2, k)])
              clauses.append([-encodeAtom(1, i1, j1, k), -encodeAtom(1, i2, j2, k)])

    for i in range(height):
      for j in range(width):

        if StartBoard[i][j] == 1:
          clauses.append([-encodeAtom(0, i, j, step_number)])
          clauses.append([-encodeAtom(1, i, j, step_number)])

        clauses.append([-encodeAtom(0, i, j, step_number) , -encodeAtom(1, i, j, step_number)])
        if step_number != 0:
          clauses.append([-encodeAtom(0, i, j, step_number), encodeAtom(0, i + 1, j, step_number - 1), encodeAtom(0, i - 1, j, step_number - 1),
                          encodeAtom(0, i, j + 1, step_number - 1), encodeAtom(0, i, j - 1, step_number - 1)])
        clauses.append([-encodeAtom(0, i, j, step_number), encodeAtom(0, i + 1, j, step_number + 1), encodeAtom(0, i - 1, j, step_number + 1),
                        encodeAtom(0, i, j + 1, step_number + 1), encodeAtom(0, i, j - 1, step_number + 1)])

        l1 = [encodeAtom(1, i - 1, j, k - 1), encodeAtom(0, i - 2, j, k - 1), encodeAtom(0, i - 1, j, k)]
        l2 = [encodeAtom(1, i + 1, j, k - 1), encodeAtom(0, i + 2, j, k - 1), encodeAtom(0, i + 1, j, k)]
        l3 = [encodeAtom(1, i, j - 1, k - 1), encodeAtom(0, i, j - 2, k - 1), encodeAtom(0, i, j - 1, k)]
        l4 = [encodeAtom(1, i, j + 1, k - 1), encodeAtom(0, i, j + 2, k - 1), encodeAtom(0, i, j + 1, k)]

        if step_number != 0:
          for ind1 in range(3):
            for ind2 in range(3):
              for ind3 in range(3):
                for ind4 in range(3):
                  clauses.append([-encodeAtom(1, i, j, k), encodeAtom(1, i, j, k - 1), l1[ind1], l2[ind2], l3[ind3], l4[ind4]])


    return clauses

# Solution Decoding
Extract and display the solution from the solver's results.

In [9]:
def print_solution(model,StartBoard,number_of_steps):

    height = len(StartBoard)
    width = len(StartBoard[0])

    print('\n## Solution:')
    print('\n Number of steps: ' + str(number_of_steps) + '\n')
    for t in range(number_of_steps+1):
        print("\n STEP " + str(t) + ' Board:')

        # deepCopy will copy Board in 2D array (pass by value)
        A = copy.deepcopy(StartBoard)
        for i in range(len(A)):
          for j in range(len(A[i])):
            if A[i][j] == 'p' or A[i][j] == 'b':
              A[i][j] = 0

        for phi in model:
          atom = decodeAtom(phi)
          if phi >= 0 and atom[3] == t and min(atom[1], atom[2]) >= 0 and atom[1] < height and atom[2] < width:
            if atom[0] == 0:
              A[atom[1]][atom[2]] = 'p'
            elif atom[0] == 1:
              A[atom[1]][atom[2]] = 'b'

        for i in range(len(A)):
          for j in range(len(A[i])):
            print(A[i][j], end=' ')
          print()

    print('\n## END of solution')
    return True

# Solver Implementation
Develop the main function to solve the puzzle using SAT logic.

In [10]:
def solve_sokoban(StartBoard, goal_position, max_steps):

    clauses = CNF()
    clauses = create_initial_clauses(clauses, StartBoard)

    height = len(StartBoard)
    width = len(StartBoard[0])


    for i in range(-1, height + 1):
      for j in range(-1, width + 1):
        for s in range(-1, max_steps + 1):
          if min(i, j, s) == -1 or i >= height or j >= width:
            clauses.append([-encodeAtom(0, i, j, s)])
            clauses.append([-encodeAtom(1, i, j, s)])

    for s in range(0, max_steps + 1):
      create_transition_clauses(clauses, StartBoard, s)
      create_Final_clause(clauses, goal_position, s)
      solver = Solver(bootstrap_with=clauses)

      if solver.solve():
        model = solver.get_model()
        print_solution(model, StartBoard, s)
        break

      Delete_Final_clause(clauses, goal_position, s)
      solver.delete()
    else:
      print("No solution found")

    return None

# Game Setup
Create and solve Sokoban puzzles using the implemented solver.

In [11]:
if __name__ == '__main__':

    goal_position = (3, 4)
    StartBoard = [
        [0, 0, 0, 0, 0],
        ['p', 'b', 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0]
    ]

    max_steps = 11

    solve_sokoban(StartBoard, goal_position, max_steps)


## Solution:

 Number of steps: 11


 STEP 0 Board:
0 0 0 0 0 
p b 1 0 0 
0 0 0 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 1 Board:
p 0 0 0 0 
0 b 1 0 0 
0 0 0 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 2 Board:
0 p 0 0 0 
0 b 1 0 0 
0 0 0 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 3 Board:
0 0 0 0 0 
0 p 1 0 0 
0 b 0 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 4 Board:
0 0 0 0 0 
p 0 1 0 0 
0 b 0 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 5 Board:
0 0 0 0 0 
0 0 1 0 0 
p b 0 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 6 Board:
0 0 0 0 0 
0 0 1 0 0 
0 p b 0 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 7 Board:
0 0 0 0 0 
0 0 1 0 0 
0 0 p b 0 
0 0 1 0 0 
0 0 1 0 0 

 STEP 8 Board:
0 0 0 0 0 
0 0 1 0 0 
0 0 0 p b 
0 0 1 0 0 
0 0 1 0 0 

 STEP 9 Board:
0 0 0 0 0 
0 0 1 p 0 
0 0 0 0 b 
0 0 1 0 0 
0 0 1 0 0 

 STEP 10 Board:
0 0 0 0 0 
0 0 1 0 p 
0 0 0 0 b 
0 0 1 0 0 
0 0 1 0 0 

 STEP 11 Board:
0 0 0 0 0 
0 0 1 0 0 
0 0 0 0 p 
0 0 1 0 b 
0 0 1 0 0 

## END of solution
